In [1]:
import os
import sys

In [2]:
#os.chdir('/home/au643300/DataHandling/')
#sys.path.append('/home/au643300/DataHandling/')
os.environ['HDF5_USE_FILE_LOCKING']="FALSE"

In [3]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, as_completed,wait,fire_and_forget,progress
import glob
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import importlib
import dask
import zarr
import time
from rechunker import rechunk

In [4]:
def custom_optimize(dsk, keys):
    dsk = dask.optimization.inline(dsk, inline_constants=True)
    return dask.array.optimization.optimize(dsk, keys)
sns.set_theme()
dask.config.set({"distributed.comm.timeouts.tcp": "50s"})
dask.config.set({'distributed.comm.retry.count':3})
dask.config.set({'distributed.comm.timeouts.connect':'25s'})
dask.config.set({"distributed.worker.use-file-locking":False})
zarr.blosc.use_threads = False

Loads the Datahandling package(which is everything in the src folder)

In [5]:
%load_ext autoreload


In [6]:
%autoreload 2

In [7]:
import importlib
from DataHandling.data.toxarr import to_xarr

In [8]:
cluster=SLURMCluster(cores=8,
                     memory="31GiB",
                     processes=2,
                     queue='q64',
                     walltime='0-01:00:00',
                     local_directory='/scratch/$SLURM_JOB_ID',
                     interface='ib0',
                     scheduler_options={'interface':'ib0'},
                     extra=['--resources save=1',"--lifetime", "50m", "--lifetime-stagger", "4m"]
                    )
                    

In [9]:
client=Client(cluster)

In [10]:
cluster.adapt(minimum=2,maximum=8)

In [11]:
client

Client Scheduler: tcp://10.100.0.1:44268 Dashboard: http://10.100.0.1:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [37]:
# test="/home/au643300/NOBACKUP/interim/"
# files=glob.glob(test+'*.nc')
# files=sorted(files)
# ex=xr.open_mfdataset(files[0:2],parallel=True,combine='nested',concat_dim='time')
# ex.attrs['field']=files[1][-7:-3]
# ex.to_zarr("/home/au643300/NOBACKUP/data.zarr")


In [38]:
ex=xr.open_zarr("/home/au643300/NOBACKUP/data.zarr")
ex

<xarray.Dataset>
Dimensions:  (time: 2, x: 256, y: 201, z: 256)
Coordinates:
  * time     (time) float64 1.996e+03 2e+03
  * x        (x) float64 0.0 0.04688 0.09375 0.1406 ... 11.81 11.86 11.91 11.95
  * y        (y) float64 2.0 2.0 2.0 1.999 ... 0.00111 0.0004934 0.0001234 0.0
  * z        (z) float64 -3.0 -2.977 -2.953 -2.93 ... 2.906 2.93 2.953 2.977
Data variables:
    pr0.025  (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr0.2    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr0.71   (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr1      (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    u_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    v_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    w_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
Attributes:
    field:    0494

In [39]:
raw="/home/au643300/DataHandling/data/raw/"
files=glob.glob(raw+'*.u')
files=sorted(files)
file_names=[os.path.basename(path) for path in files]
file_names=[file[0:-2] for file in file_names]

In [40]:
field='field.'+ex.attrs['field']
field

'field.0494'

In [41]:
start=[field==file for file in file_names]
index=int(np.where(start)[0])
index+=1
index

2

In [43]:
for file in files[index:]:
    data=to_xarr(file)
    data.attrs['field']=file[-6:-2]
    data.to_zarr("/home/au643300/NOBACKUP/data.zarr", append_dim="time",compute=True)
    del data

done with /home/au643300/DataHandling/data/raw/field.0495.u
done with /home/au643300/DataHandling/data/raw/field.0496.u
done with /home/au643300/DataHandling/data/raw/field.0497.u


KeyboardInterrupt: 

In [35]:
ds=xr.open_zarr("/home/au643300/NOBACKUP/data.zarr")
ds

<xarray.Dataset>
Dimensions:  (time: 17, x: 256, y: 201, z: 256)
Coordinates:
  * time     (time) float64 1.996e+03 2e+03 2.012e+03 ... 2.044e+03 2.048e+03
  * x        (x) float64 0.0 0.04688 0.09375 0.1406 ... 11.81 11.86 11.91 11.95
  * y        (y) float64 2.0 2.0 2.0 1.999 ... 0.00111 0.0004934 0.0001234 0.0
  * z        (z) float64 -3.0 -2.977 -2.953 -2.93 ... 2.906 2.93 2.953 2.977
Data variables:
    pr0.025  (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr0.2    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr0.71   (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr1      (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    u_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    v_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    w_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>